# CUTLASS概述 - 实践

> 通过实际代码理解CUTLASS和Tensor Core

---

本notebook将通过Python代码帮助你理解：
1. 为什么矩阵乘法需要优化
2. Tensor Core的加速效果
3. CUTLASS相比于cuBLAS的灵活性


## 1. 环境检查


In [ ]:
import torch
import time
import numpy as np

print("="*50)
print("环境信息")
print("="*50)
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    props = torch.cuda.get_device_properties(0)
    print(f"GPU: {props.name}")
    print(f"计算能力: SM{props.major}{props.minor}")
    print(f"显存: {props.total_memory / 1e9:.1f} GB")
    
    # 检查Tensor Core支持
    if props.major >= 7:
        print("✓ 支持Tensor Core")
    else:
        print("✗ 不支持Tensor Core (需要Volta架构及以上)")
else:
    print("警告: 未检测到CUDA，部分实验无法运行")


## 2. 矩阵乘法性能分析

让我们通过测量矩阵乘法的性能来理解为什么需要Tensor Core。


In [ ]:
def benchmark_matmul(M, N, K, dtype=torch.float16, warmup=10, iterations=100):
    """
    测量矩阵乘法性能
    C[M,N] = A[M,K] @ B[K,N]
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # 创建随机矩阵
    A = torch.randn(M, K, dtype=dtype, device=device)
    B = torch.randn(K, N, dtype=dtype, device=device)
    
    # 预热
    for _ in range(warmup):
        C = torch.matmul(A, B)
    torch.cuda.synchronize()
    
    # 计时
    start = time.perf_counter()
    for _ in range(iterations):
        C = torch.matmul(A, B)
    torch.cuda.synchronize()
    elapsed = time.perf_counter() - start
    
    # 计算性能指标
    avg_time_ms = (elapsed / iterations) * 1000
    flops = 2 * M * N * K  # 矩阵乘法的FLOPs
    tflops = (flops / (avg_time_ms / 1000)) / 1e12
    
    return avg_time_ms, tflops


In [ ]:
if torch.cuda.is_available():
    print("矩阵乘法性能测试 (C = A @ B)")
    print("="*70)
    print(f"{'Size (M=N=K)':<15} {'FP32 Time (ms)':<18} {'FP16 Time (ms)':<18} {'FP16 TFLOPS':<15}")
    print("-"*70)
    
    sizes = [512, 1024, 2048, 4096]
    
    for size in sizes:
        # FP32 (不使用Tensor Core)
        time_fp32, _ = benchmark_matmul(size, size, size, dtype=torch.float32, iterations=50)
        
        # FP16 (使用Tensor Core)
        time_fp16, tflops_fp16 = benchmark_matmul(size, size, size, dtype=torch.float16, iterations=50)
        
        speedup = time_fp32 / time_fp16
        print(f"{size:<15} {time_fp32:<18.3f} {time_fp16:<18.3f} {tflops_fp16:<15.1f}")
    
    print("\n说明:")
    print("- FP16自动使用Tensor Core (在支持的GPU上)")
    print("- Tensor Core可以带来显著的加速")
else:
    print("需要CUDA环境运行此测试")


## 3. 理解计算强度

计算强度（Arithmetic Intensity）决定了算法是内存绑定还是计算绑定。


In [ ]:
def analyze_gemm_intensity(M, N, K, dtype_bytes=2):
    """
    分析GEMM的计算强度
    
    C[M,N] = A[M,K] @ B[K,N]
    
    FLOPs = 2 * M * N * K
    Memory = (M*K + K*N + M*N) * dtype_bytes
    """
    flops = 2 * M * N * K
    memory_bytes = (M*K + K*N + M*N) * dtype_bytes
    intensity = flops / memory_bytes
    
    return flops, memory_bytes, intensity

print("GEMM计算强度分析")
print("="*70)
print(f"{'M=N=K':<10} {'FLOPs (G)':<15} {'Memory (MB)':<15} {'Intensity':<15}")
print("-"*70)

for size in [256, 512, 1024, 2048, 4096, 8192]:
    flops, mem, intensity = analyze_gemm_intensity(size, size, size, dtype_bytes=2)
    print(f"{size:<10} {flops/1e9:<15.2f} {mem/1e6:<15.2f} {intensity:<15.1f}")

print("\n解读:")
print("- 计算强度 = FLOPs / Memory访问字节数")
print("- A100平衡点约为 156 (312 TFLOPS / 2 TB/s)")
print("- 大矩阵的计算强度更高，更能利用Tensor Core")


## 4. Attention的计算强度分析

让我们分析标准Attention各步骤的计算强度。


In [ ]:
def analyze_attention_intensity(N, d, dtype_bytes=2):
    """
    分析Attention各步骤的计算强度
    
    Step 1: S = Q @ K^T   [N,d] @ [d,N] -> [N,N]
    Step 2: P = softmax(S)  [N,N] -> [N,N]
    Step 3: O = P @ V     [N,N] @ [N,d] -> [N,d]
    """
    results = []
    
    # Step 1: S = Q @ K^T
    flops1 = 2 * N * N * d
    mem1 = (N*d + N*d + N*N) * dtype_bytes  # Q + K + S
    intensity1 = flops1 / mem1
    results.append(("Q @ K^T", flops1, mem1, intensity1))
    
    # Step 2: softmax (approximately)
    flops2 = 5 * N * N  # exp, sum, div
    mem2 = 2 * N * N * dtype_bytes  # read S, write P
    intensity2 = flops2 / mem2
    results.append(("softmax", flops2, mem2, intensity2))
    
    # Step 3: O = P @ V
    flops3 = 2 * N * d * N
    mem3 = (N*N + N*d + N*d) * dtype_bytes  # P + V + O
    intensity3 = flops3 / mem3
    results.append(("P @ V", flops3, mem3, intensity3))
    
    return results

print("标准Attention计算强度分析")
print("(序列长度N=2048, 头维度d=64, FP16)")
print("="*70)

results = analyze_attention_intensity(2048, 64)
print(f"{'步骤':<12} {'FLOPs (M)':<15} {'Memory (MB)':<15} {'计算强度':<15}")
print("-"*70)
for step, flops, mem, intensity in results:
    print(f"{step:<12} {flops/1e6:<15.1f} {mem/1e6:<15.1f} {intensity:<15.1f}")

print("\n关键洞察:")
print("- softmax的计算强度极低 (~2.5)，严重内存绑定")
print("- 矩阵乘法的计算强度 (~64) 也低于GPU平衡点 (~156)")
print("- 这就是为什么FlashAttention需要融合这些操作！")


## 5. 模拟CUTLASS的Tile分块

CUTLASS将大矩阵分成小块(Tile)在SRAM中计算。让我们模拟这个过程。


In [ ]:
def tiled_matmul_simulation(M, N, K, tile_m=64, tile_n=64, tile_k=32):
    """
    模拟CUTLASS的分块矩阵乘法
    
    展示如何将大矩阵分成小块处理
    """
    print(f"矩阵大小: A[{M}×{K}] @ B[{K}×{N}] = C[{M}×{N}]")
    print(f"Tile大小: {tile_m}×{tile_n}×{tile_k}")
    print()
    
    # 计算需要的tile数量
    num_tiles_m = (M + tile_m - 1) // tile_m
    num_tiles_n = (N + tile_n - 1) // tile_n
    num_tiles_k = (K + tile_k - 1) // tile_k
    
    total_tiles = num_tiles_m * num_tiles_n
    inner_iterations = num_tiles_k
    
    print(f"输出Tile数量: {num_tiles_m} × {num_tiles_n} = {total_tiles}")
    print(f"每个输出Tile需要K方向迭代: {inner_iterations}次")
    print()
    
    # 计算每个Tile需要的SRAM
    tile_a_size = tile_m * tile_k * 2  # FP16
    tile_b_size = tile_k * tile_n * 2
    tile_c_size = tile_m * tile_n * 4  # FP32累加
    total_sram = tile_a_size + tile_b_size + tile_c_size
    
    print(f"每个Tile的SRAM需求:")
    print(f"  A tile: {tile_a_size/1024:.1f} KB")
    print(f"  B tile: {tile_b_size/1024:.1f} KB")
    print(f"  C tile: {tile_c_size/1024:.1f} KB")
    print(f"  总计: {total_sram/1024:.1f} KB")
    print()
    
    # A100的共享内存大小
    a100_smem = 164 * 1024  # 164 KB
    print(f"A100共享内存: {a100_smem/1024:.0f} KB")
    print(f"利用率: {total_sram/a100_smem*100:.1f}%")

# 模拟FlashAttention中的分块
print("="*60)
print("FlashAttention风格的分块 (序列长度2048, 头维度64)")
print("="*60)
tiled_matmul_simulation(2048, 64, 2048, tile_m=128, tile_n=64, tile_k=64)


## 6. 可视化分块过程


In [ ]:
def visualize_tiling():
    """
    使用ASCII艺术可视化分块过程
    """
    diagram = """
    CUTLASS分块矩阵乘法示意图
    ========================
    
    矩阵A [M×K]              矩阵B [K×N]              矩阵C [M×N]
    ┌─────┬─────┬─────┐     ┌─────┬─────┬─────┐     ┌─────┬─────┬─────┐
    │ A00 │ A01 │ A02 │     │ B00 │ B01 │ B02 │     │ C00 │ C01 │ C02 │
    ├─────┼─────┼─────┤     ├─────┼─────┼─────┤     ├─────┼─────┼─────┤
    │ A10 │ A11 │ A12 │  @  │ B10 │ B11 │ B12 │  =  │ C10 │ C11 │ C12 │
    ├─────┼─────┼─────┤     ├─────┼─────┼─────┤     ├─────┼─────┼─────┤
    │ A20 │ A21 │ A22 │     │ B20 │ B21 │ B22 │     │ C20 │ C21 │ C22 │
    └─────┴─────┴─────┘     └─────┴─────┴─────┘     └─────┴─────┴─────┘
    
    计算C11的过程:
    ─────────────────
    
    C11 = A10 @ B01 + A11 @ B11 + A12 @ B21
    
    步骤1: 加载A10, B01到SRAM → 计算A10 @ B01 → 累加到C11
    步骤2: 加载A11, B11到SRAM → 计算A11 @ B11 → 累加到C11  
    步骤3: 加载A12, B21到SRAM → 计算A12 @ B21 → 累加到C11
    
    关键优化:
    ─────────
    1. 每个Tile只从HBM加载一次
    2. 所有计算在SRAM中完成
    3. 使用异步拷贝隐藏延迟
    4. Tensor Core加速Tile内的矩阵乘法
    """
    print(diagram)

visualize_tiling()


## 7. CUTLASS vs 朴素实现的对比

让我们对比一个朴素的分块实现和PyTorch（使用cuBLAS/Tensor Core）的性能差距。


In [ ]:
def naive_tiled_matmul(A, B, tile_size=64):
    """
    朴素的分块矩阵乘法（纯Python/PyTorch，不使用优化）
    用于演示分块的概念
    """
    M, K = A.shape
    K2, N = B.shape
    assert K == K2
    
    C = torch.zeros(M, N, dtype=A.dtype, device=A.device)
    
    # 分块遍历
    for i in range(0, M, tile_size):
        for j in range(0, N, tile_size):
            for k in range(0, K, tile_size):
                # 获取tile
                i_end = min(i + tile_size, M)
                j_end = min(j + tile_size, N)
                k_end = min(k + tile_size, K)
                
                # Tile乘法并累加
                C[i:i_end, j:j_end] += A[i:i_end, k:k_end] @ B[k:k_end, j:j_end]
    
    return C

if torch.cuda.is_available():
    print("分块矩阵乘法演示")
    print("="*50)
    
    M, N, K = 512, 512, 512
    A = torch.randn(M, K, dtype=torch.float16, device='cuda')
    B = torch.randn(K, N, dtype=torch.float16, device='cuda')
    
    # 正确性验证
    C_torch = A @ B
    C_tiled = naive_tiled_matmul(A, B, tile_size=64)
    
    max_diff = (C_torch - C_tiled).abs().max().item()
    print(f"最大误差: {max_diff:.6f}")
    print(f"结果正确: {max_diff < 0.01}")
    print()
    print("注意: 这个朴素实现只是演示分块概念")
    print("CUTLASS的实现要复杂得多，包含:")
    print("  - 共享内存优化")
    print("  - Tensor Core调用")
    print("  - 异步拷贝")
    print("  - 软件流水线")
else:
    print("需要CUDA环境运行此测试")


## 8. 总结

通过这个notebook，我们理解了：

1. **Tensor Core的重要性**: FP16矩阵乘法可以获得显著加速
2. **计算强度**: Attention操作是内存绑定的，需要优化
3. **分块策略**: 将大矩阵分成适合SRAM的小块
4. **CUTLASS的价值**: 提供了灵活的分块和Tensor Core封装

下一步：学习cute张量抽象层，了解如何用现代方式表达这些操作。

---

## 📚 延伸阅读

- [CUTLASS Profiler](https://github.com/NVIDIA/cutlass/blob/main/tools/profiler/README.md): 用于性能分析
- [CUTLASS Examples](https://github.com/NVIDIA/cutlass/tree/main/examples): 官方示例代码
- [Nsight Compute](https://developer.nvidia.com/nsight-compute): NVIDIA性能分析工具
